# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 26 2022 2:27PM,242781,20,ALUR950767096,Alumier Labs Inc.,Released
1,Jul 26 2022 2:27PM,242781,20,ALUR639932385,Alumier Labs Inc.,Released
2,Jul 26 2022 2:27PM,242781,20,ALU0007366,Alumier Labs Inc.,Released
3,Jul 26 2022 2:27PM,242781,20,ALUR007877657,Alumier Labs Inc.,Released
4,Jul 26 2022 2:27PM,242781,20,ALUR082074284,Alumier Labs Inc.,Released
5,Jul 26 2022 2:27PM,242781,20,ALUR132615379,Alumier Labs Inc.,Released
6,Jul 26 2022 2:27PM,242781,20,ALUR188015391,Alumier Labs Inc.,Released
7,Jul 26 2022 2:27PM,242781,20,ALUR378245188,Alumier Labs Inc.,Released
8,Jul 26 2022 2:27PM,242781,20,ALUR461133250,Alumier Labs Inc.,Released
9,Jul 26 2022 2:27PM,242781,20,ALUR483999395,Alumier Labs Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
33,242776,Released,1
34,242777,Released,1
35,242779,Released,4
36,242780,Released,2
37,242781,Released,14


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
242776,NaN,NaN,1.0
242777,NaN,NaN,1.0
242779,NaN,NaN,4.0
242780,NaN,NaN,2.0
242781,NaN,NaN,14.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242718,10.0,27.0,2.0
242720,0.0,0.0,50.0
242721,9.0,5.0,0.0
242723,0.0,5.0,26.0
242730,0.0,6.0,3.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242718,10,27,2
242720,0,0,50
242721,9,5,0
242723,0,5,26
242730,0,6,3


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242718,10,27,2
1,242720,0,0,50
2,242721,9,5,0
3,242723,0,5,26
4,242730,0,6,3


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242718,10,27,2
1,242720,,,50
2,242721,9,5,
3,242723,,5,26
4,242730,,6,3


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 26 2022 2:27PM,242781,20,Alumier Labs Inc.
14,Jul 26 2022 2:23PM,242777,15,"Carwin Pharmaceutical Associates, LLC"
15,Jul 26 2022 2:19PM,242754,20,"ACI Healthcare USA, Inc."
27,Jul 26 2022 2:19PM,242780,10,ISDIN Corporation
29,Jul 26 2022 2:17PM,242779,10,Eye Pharma Inc
33,Jul 26 2022 1:59PM,242776,10,Yusen – 3D Matrix
34,Jul 26 2022 1:55PM,242775,10,Yusen – 3D Matrix
35,Jul 26 2022 1:55PM,242773,10,"Citieffe, Inc."
36,Jul 26 2022 1:53PM,242774,10,Yusen – 3D Matrix
37,Jul 26 2022 1:44PM,242748,20,"ACI Healthcare USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 26 2022 2:27PM,242781,20,Alumier Labs Inc.,,,14
1,Jul 26 2022 2:23PM,242777,15,"Carwin Pharmaceutical Associates, LLC",,,1
2,Jul 26 2022 2:19PM,242754,20,"ACI Healthcare USA, Inc.",,,12
3,Jul 26 2022 2:19PM,242780,10,ISDIN Corporation,,,2
4,Jul 26 2022 2:17PM,242779,10,Eye Pharma Inc,,,4
5,Jul 26 2022 1:59PM,242776,10,Yusen – 3D Matrix,,,1
6,Jul 26 2022 1:55PM,242775,10,Yusen – 3D Matrix,,,1
7,Jul 26 2022 1:55PM,242773,10,"Citieffe, Inc.",,,1
8,Jul 26 2022 1:53PM,242774,10,Yusen – 3D Matrix,,,1
9,Jul 26 2022 1:44PM,242748,20,"ACI Healthcare USA, Inc.",,13,27


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 26 2022 2:27PM,242781,20,Alumier Labs Inc.,14,,
1,Jul 26 2022 2:23PM,242777,15,"Carwin Pharmaceutical Associates, LLC",1,,
2,Jul 26 2022 2:19PM,242754,20,"ACI Healthcare USA, Inc.",12,,
3,Jul 26 2022 2:19PM,242780,10,ISDIN Corporation,2,,
4,Jul 26 2022 2:17PM,242779,10,Eye Pharma Inc,4,,
5,Jul 26 2022 1:59PM,242776,10,Yusen – 3D Matrix,1,,
6,Jul 26 2022 1:55PM,242775,10,Yusen – 3D Matrix,1,,
7,Jul 26 2022 1:55PM,242773,10,"Citieffe, Inc.",1,,
8,Jul 26 2022 1:53PM,242774,10,Yusen – 3D Matrix,1,,
9,Jul 26 2022 1:44PM,242748,20,"ACI Healthcare USA, Inc.",27,13,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 26 2022 2:27PM,242781,20,Alumier Labs Inc.,14,,
1,Jul 26 2022 2:23PM,242777,15,"Carwin Pharmaceutical Associates, LLC",1,,
2,Jul 26 2022 2:19PM,242754,20,"ACI Healthcare USA, Inc.",12,,
3,Jul 26 2022 2:19PM,242780,10,ISDIN Corporation,2,,
4,Jul 26 2022 2:17PM,242779,10,Eye Pharma Inc,4,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 26 2022 2:27PM,242781,20,Alumier Labs Inc.,14.0,NaN,NaN
1,Jul 26 2022 2:23PM,242777,15,"Carwin Pharmaceutical Associates, LLC",1.0,NaN,NaN
2,Jul 26 2022 2:19PM,242754,20,"ACI Healthcare USA, Inc.",12.0,NaN,NaN
3,Jul 26 2022 2:19PM,242780,10,ISDIN Corporation,2.0,NaN,NaN
4,Jul 26 2022 2:17PM,242779,10,Eye Pharma Inc,4.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 26 2022 2:27PM,242781,20,Alumier Labs Inc.,14.0,0.0,0.0
1,Jul 26 2022 2:23PM,242777,15,"Carwin Pharmaceutical Associates, LLC",1.0,0.0,0.0
2,Jul 26 2022 2:19PM,242754,20,"ACI Healthcare USA, Inc.",12.0,0.0,0.0
3,Jul 26 2022 2:19PM,242780,10,ISDIN Corporation,2.0,0.0,0.0
4,Jul 26 2022 2:17PM,242779,10,Eye Pharma Inc,4.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2913168,21.0,22.0,3.0
15,728240,5.0,6.0,1.0
16,485442,0.0,10.0,18.0
19,242747,0.0,15.0,4.0
20,1699209,132.0,47.0,10.0
22,485521,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2913168,21.0,22.0,3.0
1,15,728240,5.0,6.0,1.0
2,16,485442,0.0,10.0,18.0
3,19,242747,0.0,15.0,4.0
4,20,1699209,132.0,47.0,10.0
5,22,485521,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,21.0,22.0,3.0
1,15,5.0,6.0,1.0
2,16,0.0,10.0,18.0
3,19,0.0,15.0,4.0
4,20,132.0,47.0,10.0
5,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,21.0
1,15,Released,5.0
2,16,Released,0.0
3,19,Released,0.0
4,20,Released,132.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,22
Status,,,,,,
Completed,3.0,1.0,18.0,4.0,10.0,0.0
Executing,22.0,6.0,10.0,15.0,47.0,0.0
Released,21.0,5.0,0.0,0.0,132.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,22
0,Completed,3.0,1.0,18.0,4.0,10.0,0.0
1,Executing,22.0,6.0,10.0,15.0,47.0,0.0
2,Released,21.0,5.0,0.0,0.0,132.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,22
0,Completed,3.0,1.0,18.0,4.0,10.0,0.0
1,Executing,22.0,6.0,10.0,15.0,47.0,0.0
2,Released,21.0,5.0,0.0,0.0,132.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()